## Trace bedrock SDK inferences on Maxim

This notebook demonstrates how to integrate Maxim with AWS Bedrock to trace and monitor LLM inferences.
It shows how to:
1. Set up environment variables for Maxim and AWS Bedrock
2. Initialize the AWS Bedrock client
3. Send prompts to Claude models via Bedrock
4. Capture and log these interactions with Maxim for monitoring and analysis

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access environment variables
MAXIM_API_KEY = os.getenv('MAXIM_API_KEY')
MAXIM_LOG_REPO_ID = os.getenv('MAXIM_LOG_REPO_ID')
AWS_MODEL_ARN = os.getenv('AWS_MODEL_ARN')
AWS_REGION = os.getenv('AWS_REGION')
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

## Initialize Maxim SDK

In [2]:
from maxim import Maxim,Config
from maxim.logger import SessionConfig
# Creating a new logger instance
# It automatically initializes using MAXIM_API_KEY and MAXIM_LOG_REPO_ID from env variables.
logger = Maxim(Config(raise_exceptions=False)).logger()

## Initialize boto

In [3]:
import boto3
from maxim.logger.bedrock import MaximBedrockClient
model_arn = AWS_MODEL_ARN 
bedrock_client = boto3.client('bedrock-runtime', region_name=AWS_REGION, aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
maxim_bedrock_client = MaximBedrockClient(logger, bedrock_client)

Initializing MaximBedrockAsyncClient


## Get response

In [4]:
prompt = "Who was the first president of the United States? and where was he born?"

# Prepare messages in the Converse API format
messages = [
    {
        "role": "user",
        "content": [{"text": prompt}],  # Content is a list, use {"text": ...} for text
    }
]
inference_config = {"maxTokens": 100, "temperature": 0.7}

try:
    # No need for await here since we're using the synchronous client
    response_dict = maxim_bedrock_client.converse(
        messages=messages,
        modelId=model_arn,
        inferenceConfig=inference_config,
        # Add other parameters like system if needed
    )
    print("\n Non-streaming response received:", response_dict, "\n")    
except Exception as e:
    print(f"Error during non-streaming call: {type(e).__name__} - {e}")


 Non-streaming response received: {'ResponseMetadata': {'RequestId': 'e6fbe133-4795-4fff-8e88-d6bdee51e823', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 19 Apr 2025 19:33:26 GMT', 'content-type': 'application/json', 'content-length': '679', 'connection': 'keep-alive', 'x-amzn-requestid': 'e6fbe133-4795-4fff-8e88-d6bdee51e823'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'George Washington was the first president of the United States, serving from 1789 to 1797. He was born on February 22, 1732, in Westmoreland County, Virginia, on a plantation along the Potomac River. He is often referred to as the "Father of His Country" for his crucial role in the founding of the United States and leading the Continental Army during the American Revolutionary War.'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 23, 'outputTokens': 91, 'totalTokens': 114}, 'metrics': {'latencyMs': 1765}} 

